## NYC MTA Turnstile Usage Dataset Analysis by Andy Chen

Have a look at / download the following dataset :
- Go to http://web.mta.info/developers/turnstile.html

- This dataset shows entry & exit counter values for each turnstile-device in each station in the NYC Subway System.

- Note these aren’t counts per interval, but equivalent to an “odometer” reading for each device.
*You will be evaluated in terms of coding, data analysis, and visualization skills.

Data analysis:
- Which station has the most number of units?

- What is the total number of entries & exits across the subway system for February 1, 2013?

- Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on February 1, 2013? What turnstile was the busiest on that date?

- What stations have seen the most usage growth/decline in 2013?

- What dates are the least busy? Could you identify days on which stations were not operating at full capacity or closed entirely?

- Bonus:  What hour is the busiest for station CANAL ST in Q1 2013?

Visualization:
- Plot the daily row counts for data files in Q1 2013.

- Plot the daily total number of entries & exits across the system for Q1 2013.

- Plot the mean and standard deviation of the daily total number of entries & exits for each month in Q1 2013 for station 34 ST-PENN STA.

- Plot 25/50/75 percentile of the daily total number of entries & exits for each month in Q1 2013 for station 34 ST-PENN STA.

- Plot the daily number of closed stations and number of stations that were not operating at full capacity in Q1 2013.

In [1]:
import pandas as pd
df_unit=pd.read_excel('http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls')
print(df_unit.groupby('Station').Remote.nunique().reset_index(name='unit_count').sort_values(['unit_count'],ascending=False).head(10))

             Station  unit_count
114            86 ST           5
175         CANAL ST           5
73   42 ST-GRD CNTRL           4
68    34 ST-PENN STA           4
231        FULTON ST           4
11            125 ST           4
338     PROSPECT AVE           3
91             59 ST           3
269    KINGS HIGHWAY           3
84             50 ST           3


In [5]:
import pandas as pd
df130202=pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_130202.txt', header=None)
#Rename columns
df130202.columns = [ 'C/A','UNIT','SCP'
                    ,'DATE1','TIME1','DESC1','ENTRIES1','EXITS1'
                    ,'DATE2','TIME2','DESC2','ENTRIES2','EXITS2'
                    ,'DATE3','TIME3','DESC3','ENTRIES3','EXITS3'
                    ,'DATE4','TIME4','DESC4','ENTRIES4','EXITS4'
                    ,'DATE5','TIME5','DESC5','ENTRIES5','EXITS5'
                    ,'DATE6','TIME6','DESC6','ENTRIES6','EXITS6'
                    ,'DATE7','TIME7','DESC7','ENTRIES7','EXITS7'
                    ,'DATE8','TIME8','DESC8','ENTRIES8','EXITS8' ]
#Restructure dataframes
df1=df130202[['C/A','UNIT','SCP','DATE1','TIME1','DESC1','ENTRIES1','EXITS1']]
df2=df130202[['C/A','UNIT','SCP','DATE2','TIME2','DESC2','ENTRIES2','EXITS2']]
df3=df130202[['C/A','UNIT','SCP','DATE3','TIME3','DESC3','ENTRIES3','EXITS3']]
df4=df130202[['C/A','UNIT','SCP','DATE4','TIME4','DESC4','ENTRIES4','EXITS4']]
df5=df130202[['C/A','UNIT','SCP','DATE5','TIME5','DESC5','ENTRIES5','EXITS5']]
df6=df130202[['C/A','UNIT','SCP','DATE6','TIME6','DESC6','ENTRIES6','EXITS6']]
df7=df130202[['C/A','UNIT','SCP','DATE7','TIME7','DESC7','ENTRIES7','EXITS7']]
df8=df130202[['C/A','UNIT','SCP','DATE8','TIME8','DESC8','ENTRIES8','EXITS8']]
#Rename dataframes
df1.columns=['C/A','UNIT','SCP','DATE','TIME','DESC','ENTRIES','EXITS']
df2.columns=['C/A','UNIT','SCP','DATE','TIME','DESC','ENTRIES','EXITS']
df3.columns=['C/A','UNIT','SCP','DATE','TIME','DESC','ENTRIES','EXITS']
df4.columns=['C/A','UNIT','SCP','DATE','TIME','DESC','ENTRIES','EXITS']
df5.columns=['C/A','UNIT','SCP','DATE','TIME','DESC','ENTRIES','EXITS']
df6.columns=['C/A','UNIT','SCP','DATE','TIME','DESC','ENTRIES','EXITS']
df7.columns=['C/A','UNIT','SCP','DATE','TIME','DESC','ENTRIES','EXITS']
df8.columns=['C/A','UNIT','SCP','DATE','TIME','DESC','ENTRIES','EXITS']
#Reshape the data
df130202rs=df1.append(df2)
df130202rs=df130202rs.append(df3)
df130202rs=df130202rs.append(df4)
df130202rs=df130202rs.append(df5)
df130202rs=df130202rs.append(df6)
df130202rs=df130202rs.append(df7)
df130202rs=df130202rs.append(df8)
#Now we can format our data as below:
df=df130202rs.sort_values(['C/A','UNIT','SCP','DATE','TIME']).dropna()

a=df.groupby(['C/A','UNIT','SCP','DATE'])['ENTRIES','EXITS'].min()
b=df.groupby(['C/A','UNIT','SCP','DATE'])['ENTRIES','EXITS'].max()
c=b-a
d=c.groupby('DATE')[['ENTRIES','EXITS']].sum()
print(d)

              ENTRIES         EXITS
DATE                               
01-26-13    9299906.0  3.658605e+06
01-27-13    9810169.0  8.205277e+06
01-28-13    5162636.0  1.982119e+07
01-29-13   56099709.0  2.131601e+07
01-30-13    5722425.0  4.364807e+06
01-31-13  823381784.0  1.061253e+09
02-01-13   90830543.0  9.565364e+06


In [65]:
import pandas as pd
df_key = pd.read_excel('http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls')
df_key.columns=['UNIT', 'C/A', 'STATION', 'LINE','DIVISION']
e = pd.merge(df,df_key,on=['UNIT','C/A'],how = 'left')
f=e[['DATE','TIME','STATION','UNIT','SCP','ENTRIES','EXITS']].set_index('DATE')
g=f.loc['02-01-13']
h=g.groupby(['STATION','UNIT','SCP'])['ENTRIES','EXITS'].min()
i=g.groupby(['STATION','UNIT','SCP'])['ENTRIES','EXITS'].max()
j=i-h
j['BUSYNESS']=j['ENTRIES']+j['EXITS']
k=j.groupby('STATION').sum()
l=k.sort_values(['BUSYNESS'],ascending=False).head(5)
print(l)
m=j.groupby('UNIT').sum()
n=m.sort_values(['BUSYNESS'],ascending=False).head(5)
print(n)

                    ENTRIES      EXITS    BUSYNESS
STATION                                           
BEDFORD/NOSTRAN  80911845.0  3010149.0  83921994.0
JOURNAL SQUARE    4453899.0  2371677.0   6825576.0
34 ST-PENN STA     173953.0   160065.0    334018.0
42 ST-GRD CNTRL    157337.0   154075.0    311412.0
34 ST-HERALD SQ    115738.0   101705.0    217443.0
         ENTRIES      EXITS    BUSYNESS
UNIT                                   
R269  80911845.0  3010149.0  83921994.0
R552   4453899.0  2371677.0   6825576.0
R170     89637.0    90554.0    180191.0
R046     78900.0    72773.0    151673.0
R020     63287.0    64097.0    127384.0
